In [3]:
import pandas as pd
import numpy as np
import geopandas as gpd
import json
import pandas_bokeh
pandas_bokeh.output_notebook()

Loading BokehJS ...

In [4]:
# Reading shape files
region = gpd.read_file("Régions_Sngal.shp")

In [5]:
#Changing reading system
reg = region.to_crs('EPSG:4326')

In [6]:
#For coordinates generation
from functools import partial
import pyproj
from shapely.ops import transform
from shapely.geometry import Point

pas = 100 # simulisation sur un rayon de pas
Nsimul = 30 # nombre de points a simuler

proj_wgs84 = pyproj.Proj(init='epsg:4326')


def geodesic_point_buffer(lat, lon, km, Nsimul):
    # Azimuthal equidistant projection
    aeqd_proj = '+proj=aeqd +lat_0={lat} +lon_0={lon} +x_0=0 +y_0=0'
    project = partial(
        pyproj.transform,
        pyproj.Proj(aeqd_proj.format(lat=lat, lon=lon)),
        proj_wgs84)
    buf = Point(0, 0).buffer(km * 1000)  # distance in metres
    return transform(project, buf).exterior.coords[:Nsimul]

# Example

b = geodesic_point_buffer(14.693425,-17.447938,3,Nsimul)


/opt/anaconda3/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/opt/anaconda3/lib/python3.7/site-packages/pyproj/crs/crs.py:294: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(" ".join((projstring, projkwargs)))


In [7]:
#Load Data
data = pd.read_excel('Covid19SN_region_detail.xlsx')
data

,Region,Departement,confirme,supect,communautaire,long,lat
0,Dakar,DAKAR-OUEST,1,1,1,-17.473611,14.695153
1,Dakar,DAKAR-OUEST,1,1,1,-17.380761,14.695134
2,Dakar,DAKAR-OUEST,1,1,1,-17.381212,14.686276
3,Dakar,DAKAR-OUEST,1,1,1,-17.382553,14.677503
4,Dakar,DAKAR-OUEST,1,1,0,-17.384770,14.668900
...,...,...,...,...,...,...,...
97,Dakar,DAKAR-CENTRE,0,1,0,-17.469468,14.676223
98,Dakar,DAKAR-CENTRE,0,1,0,-17.471097,14.678360
99,Dakar,DAKAR-CENTRE,0,1,0,-17.472502,14.680642
100,Dakar,DAKAR-CENTRE,0,1,0,-17.473671,14.683048


In [8]:
#construct gpd for data
data_gpd = gpd.GeoDataFrame(data, geometry=gpd.points_from_xy(data.long, data.lat))
data_gpd.crs = 'EPSG:4326'

In [9]:
#Visualisation
first_layer = reg.plot_bokeh(
    figsize = (900, 800),
    color = 'LightCyan',
    show_figure = False,
    legend = "Senegal Regions",
    show_colorbar=False)

second_layer = data_gpd.plot_bokeh(
    figure = first_layer,
    color = 'red',
    legend = 'Confirme',
    size = 10,
    hovertool_string="""<h1>@Region</h1>
                       # <h3>Nombre de confirmés: @confirme </h3>""",
    #plot_data_points_size=10,
    marker = 'circle'
)


second_layer

/opt/anaconda3/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/opt/anaconda3/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


Figure(id='1002', ...)